<a href="https://colab.research.google.com/github/mpekar1/DSS-IDI-modelling/blob/main/Markov_Chain_modelling_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Clean and Transform Data

In [58]:
library(readxl)
library(dplyr)
library(stringr)
library( tidyverse)

In [68]:
# Load the transition counts and population data
transition_counts <- read_excel("Markov Chain Outputs 2024_05_24_confidentialised.xlsx", sheet = "Transition counts 2011-20 conf", skip=1)
transition_counts_2021 <- read_excel("Markov Chain Outputs 2024_05_24_confidentialised.xlsx", sheet = "Transition counts 2021 conf",skip=1)
dss_pop_by_age <- read_excel("Markov Chain Outputs 2024_05_24_confidentialised.xlsx", sheet = "Total DSS pop by age conf")
new_entrants <- read_excel("Markov Chain Outputs 2024_05_24_confidentialised.xlsx", sheet = "New entrants conf")


In [69]:
## Inspect the data

head(transition_counts)
head(transition_counts_2021)
head(dss_pop_by_age)
head(new_entrants)

Row Labels,from_state,DECEASED,DSS,DSS_EDU,DSS_EDU_T1,DSS_EMPLOYED,DSS_EMPLOYED_T1,DSS_HOSPITAL,DSS_T1,⋯,EMPLOYED,EMPLOYED_T1,HOSPITAL,NO DATA,NOT BORN YET,OT,OTHER,OVERSEAS,T1,Grand Total
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Age 0-03,DECEASED,6,S,S,S,S,S,S,S,⋯,S,S,S,S,S,S,S,S,S,9
Age 0-03,DSS,12,4485,S,S,S,S,147,S,⋯,S,S,S,249,S,S,S,12,S,4920
Age 0-03,DSS_EMPLOYED,S,S,S,S,S,S,S,S,⋯,S,S,S,S,S,S,S,S,S,S
Age 0-03,DSS_HOSPITAL,9,402,S,S,S,S,201,S,⋯,S,S,18,39,S,S,S,S,S,684
Age 0-03,DSS_T2,S,S,S,S,S,S,S,S,⋯,S,S,S,S,S,S,S,S,S,S
Age 0-03,DSS-OTHER,S,24,S,S,S,S,S,S,⋯,S,S,S,S,S,6,S,S,S,96


Row Labels,from_state,DECEASED,DSS,DSS_EDU,DSS_EDU_T1,DSS_EMPLOYED,DSS_EMPLOYED_T1,DSS_HOSPITAL,DSS_T1,⋯,EDU_EMPLOYED,EMPLOYED,EMPLOYED_T1,HOSPITAL,NO DATA,OT,OTHER,OVERSEAS,T1,Grand Total
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Age 0-03,DECEASED,S,S,S,S,S,S,S,S,⋯,S,S,S,S,S,S,S,S,S,S
Age 0-03,DSS,S,633,S,S,S,S,6,S,⋯,S,S,S,S,51,S,S,S,S,696
Age 0-03,DSS_EMPLOYED,S,S,S,S,S,S,S,S,⋯,S,S,S,S,S,S,S,S,S,S
Age 0-03,DSS_HOSPITAL,S,45,S,S,S,S,12,S,⋯,S,S,S,S,S,S,S,S,S,63
Age 0-03,DSS-OTHER,S,S,S,S,S,S,S,S,⋯,S,S,S,S,S,S,S,S,S,12
Age 0-03,EMPLOYED,S,S,S,S,S,S,S,S,⋯,S,S,S,S,S,S,S,S,S,S


age_group,refyr,total_pop
<chr>,<dbl>,<chr>
Age 11-18,2021,12255
Age 03-10,2020,10122
Age 03-10,2014,10893
Age 66+,2015,3477
Age 0-03,2020,1722
Age 19-25,2013,4848


age_group,refyr,state,new_entrants
<chr>,<dbl>,<chr>,<chr>
Age 26-35,2018,DSS_HOSPITAL,S
Age 36-45,2020,DSS_EMPLOYED,27
Age 0-03,2012,DSS-OTHER,S
Age 46-65,2019,DSS_HOSPITAL,57
Age 26-35,2011,DSS_EMPLOYED_T1,9
Age 11-18,2014,DSS_HOSPITAL,6


In [70]:
transition_counts <- transition_counts %>%
  rename(age_group = `Row Labels`)

transition_counts_2021 <- transition_counts_2021 %>%
  rename(age_group = `Row Labels`)

In [71]:
names(transition_counts)
names(transition_counts_2021)
names(dss_pop_by_age)
names(new_entrants)

[1] "age_group"       "from_state"      "DECEASED"        "DSS"            
 [5] "DSS_EDU"         "DSS_EDU_T1"      "DSS_EMPLOYED"    "DSS_EMPLOYED_T1"
 [9] "DSS_HOSPITAL"    "DSS_T1"          "DSS_T2"          "DSS-OTHER"      
[13] "EDU"             "EDU_EMPLOYED"    "EMPLOYED"        "EMPLOYED_T1"    
[17] "HOSPITAL"        "NO DATA"         "NOT BORN YET"    "OT"             
[21] "OTHER"           "OVERSEAS"        "T1"              "Grand Total"

[1] "age_group"       "from_state"      "DECEASED"        "DSS"            
 [5] "DSS_EDU"         "DSS_EDU_T1"      "DSS_EMPLOYED"    "DSS_EMPLOYED_T1"
 [9] "DSS_HOSPITAL"    "DSS_T1"          "DSS_T2"          "DSS-OTHER"      
[13] "EDU"             "EDU_EMPLOYED"    "EMPLOYED"        "EMPLOYED_T1"    
[17] "HOSPITAL"        "NO DATA"         "OT"              "OTHER"          
[21] "OVERSEAS"        "T1"              "Grand Total"

[1] "age_group" "refyr"     "total_pop"

[1] "age_group"    "refyr"        "state"        "new_entrants"

In [74]:
# Check the column names of the transition_counts dataframe
print(names(transition_counts))

 [1] "age_group"       "from_state"      "DECEASED"        "DSS"            
 [5] "DSS_EDU"         "DSS_EDU_T1"      "DSS_EMPLOYED"    "DSS_EMPLOYED_T1"
 [9] "DSS_HOSPITAL"    "DSS_T1"          "DSS_T2"          "DSS-OTHER"      
[13] "EDU"             "EDU_EMPLOYED"    "EMPLOYED"        "EMPLOYED_T1"    
[17] "HOSPITAL"        "NO DATA"         "NOT BORN YET"    "OT"             
[21] "OTHER"           "OVERSEAS"        "T1"              "Grand Total"    


In [75]:
# Function to clean transition counts data
clean_transition_data <- function(df) {
  df %>%
    mutate(across(-c(age_group, from_state), ~ replace(., . == "S", 0))) %>%
    mutate(across(-c(age_group, from_state), as.numeric)) %>%
    filter(!grepl("Total", age_group)) %>%
    select(-`Grand Total`)
}

# Function to clean DSS population and new entrants data
clean_population_data <- function(df, col_to_clean) {
  df %>%
    mutate(across(all_of(col_to_clean), ~ replace(., . == "S", 0))) %>%
    mutate(across(all_of(col_to_clean), as.numeric))
}

# Apply the function to clean the transition counts data
transition_counts <- clean_transition_data(transition_counts)
transition_counts_2021 <- clean_transition_data(transition_counts_2021)

# Apply the function to clean the DSS population by age and new entrants data
dss_pop_by_age <- clean_population_data(dss_pop_by_age, "total_pop")
new_entrants <- clean_population_data(new_entrants, "new_entrants")

# Verify the results
head(transition_counts)
head(transition_counts_2021)
head(dss_pop_by_age)
head(new_entrants)

age_group,from_state,DECEASED,DSS,DSS_EDU,DSS_EDU_T1,DSS_EMPLOYED,DSS_EMPLOYED_T1,DSS_HOSPITAL,DSS_T1,⋯,EDU_EMPLOYED,EMPLOYED,EMPLOYED_T1,HOSPITAL,NO DATA,NOT BORN YET,OT,OTHER,OVERSEAS,T1
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Age 0-03,DECEASED,6,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Age 0-03,DSS,12,4485,0,0,0,0,147,0,⋯,0,0,0,0,249,0,0,0,12,0
Age 0-03,DSS_EMPLOYED,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Age 0-03,DSS_HOSPITAL,9,402,0,0,0,0,201,0,⋯,0,0,0,18,39,0,0,0,0,0
Age 0-03,DSS_T2,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Age 0-03,DSS-OTHER,0,24,0,0,0,0,0,0,⋯,0,0,0,0,0,0,6,0,0,0


age_group,from_state,DECEASED,DSS,DSS_EDU,DSS_EDU_T1,DSS_EMPLOYED,DSS_EMPLOYED_T1,DSS_HOSPITAL,DSS_T1,⋯,EDU,EDU_EMPLOYED,EMPLOYED,EMPLOYED_T1,HOSPITAL,NO DATA,OT,OTHER,OVERSEAS,T1
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Age 0-03,DECEASED,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Age 0-03,DSS,0,633,0,0,0,0,6,0,⋯,0,0,0,0,0,51,0,0,0,0
Age 0-03,DSS_EMPLOYED,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Age 0-03,DSS_HOSPITAL,0,45,0,0,0,0,12,0,⋯,0,0,0,0,0,0,0,0,0,0
Age 0-03,DSS-OTHER,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
Age 0-03,EMPLOYED,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


age_group,refyr,total_pop
<chr>,<dbl>,<dbl>
Age 11-18,2021,12255
Age 03-10,2020,10122
Age 03-10,2014,10893
Age 66+,2015,3477
Age 0-03,2020,1722
Age 19-25,2013,4848


age_group,refyr,state,new_entrants
<chr>,<dbl>,<chr>,<dbl>
Age 26-35,2018,DSS_HOSPITAL,0
Age 36-45,2020,DSS_EMPLOYED,27
Age 0-03,2012,DSS-OTHER,0
Age 46-65,2019,DSS_HOSPITAL,57
Age 26-35,2011,DSS_EMPLOYED_T1,9
Age 11-18,2014,DSS_HOSPITAL,6


## Step 2: Calculate Historical Transition Probabilities


1. Compute Transition Counts and Probabilities:

In [77]:
# Transform the transition counts data to a long format
transition_counts_long <- transition_counts %>%
  pivot_longer(cols = -c(age_group, from_state), names_to = "to_state", values_to = "count") %>%
  filter(!is.na(count))

transition_counts_2021_long <- transition_counts_2021 %>%
  pivot_longer(cols = -c(age_group, from_state), names_to = "to_state", values_to = "count") %>%
  filter(!is.na(count))

# Compute transition probabilities
transition_probabilities <- transition_counts_long %>%
  group_by(age_group, from_state) %>%
  summarise(total_count = sum(count), .groups = 'drop') %>%
  left_join(transition_counts_long, by = c("age_group", "from_state")) %>%
  mutate(probability = count / total_count) %>%
  select(age_group, from_state, to_state, probability)




In [78]:
print(transition_probabilities)

# A tibble: 2,919 × 4
   age_group from_state to_state        probability
   <chr>     <chr>      <chr>                 <dbl>
 1 Age 0-03  DECEASED   DECEASED                  1
 2 Age 0-03  DECEASED   DSS                       0
 3 Age 0-03  DECEASED   DSS_EDU                   0
 4 Age 0-03  DECEASED   DSS_EDU_T1                0
 5 Age 0-03  DECEASED   DSS_EMPLOYED              0
 6 Age 0-03  DECEASED   DSS_EMPLOYED_T1           0
 7 Age 0-03  DECEASED   DSS_HOSPITAL              0
 8 Age 0-03  DECEASED   DSS_T1                    0
 9 Age 0-03  DECEASED   DSS_T2                    0
10 Age 0-03  DECEASED   DSS-OTHER                 0
# ℹ 2,909 more rows


## Step 3: Estimate New Entrants

1. Estimate the Total Population by Age Group:

In [79]:
# Assuming dss_pop_by_age contains the total population by age group for each year
dss_pop_by_age_summary <- dss_pop_by_age %>%
  group_by(age_group) %>%
  summarise(total_pop = sum(total_pop), .groups = 'drop')



2. Assume a Percentage for New Entrants:

In [80]:
# Assuming new_entrants contains the number of new entrants by age group for each year
new_entrants_summary <- new_entrants %>%
  group_by(age_group) %>%
  summarise(total_new_entrants = sum(new_entrants), .groups = 'drop')

# Calculate the percentage of new entrants
new_entrants_percentage <- new_entrants_summary %>%
  left_join(dss_pop_by_age_summary, by = "age_group") %>%
  mutate(percentage_new_entrants = total_new_entrants / total_pop)


3. Distribute New Entrants by State:

In [81]:
# Assuming the initial distribution of new entrants across states can be estimated from historical data
initial_state_distribution <- transition_counts_long %>%
  filter(from_state == "DSS") %>%
  group_by(age_group, to_state) %>%
  summarise(count = sum(count), .groups = 'drop') %>%
  mutate(probability = count / sum(count))

# Apply this distribution to estimate the number of new entrants by state
new_entrants_by_state <- new_entrants_summary %>%
  left_join(initial_state_distribution, by = "age_group") %>%
  mutate(new_entrants_by_state = total_new_entrants * probability) %>%
  select(age_group, to_state, new_entrants_by_state)



## Step 4: Model Initialization for Future Projection
1. Set Initial Population for Simulation

Use the population data from the last available year before 2021 (e.g., 2020) as your initial cohort.

In [82]:
initial_population_2020 <- dss_pop_by_age %>%
  filter(refyr == 2020) %>%
  select(age_group, total_pop)


2. Adjust for New Entrants:

In [ ]:
# Calculate new entrants for 2023
new_entrants_2023 <- new_entrants_percentage %>%
  left_join(total_population_by_age, by = "age_group") %>%
  mutate(NewEntrants = round(TotalPopulation * NewEntrantsPercentage / 100, na.rm = TRUE))

# Distribute new entrants by state
new_entrants_distribution <- new_entrants_2023 %>%
  left_join(new_entrants_by_state, by = c("age_group")) %>%
  mutate(NewEntrantsByState = round(NewEntrants * Proportion, na.rm = TRUE))


## Step 5: Simulate Future States
1. Apply Transition Probabilities:

In [ ]:
# Apply transition probabilities to simulate future states
future_states <- initial_population %>%
  rename(from_state = state, InitialCount = total_pop) %>%
  left_join(transition_probabilities, by = "age_group") %>%
  mutate(FutureCount = InitialCount * TransitionProbability) %>%
  group_by(age_group, to_state) %>%
  summarize(FutureCount = sum(FutureCount))


2. Iterate Annually:

In [ ]:
# For each year, apply the transition probabilities and adjust the cohort by adding new entrants
future_states_2024 <- future_states %>%
  rename(from_state = to_state, InitialCount = FutureCount) %>%
  left_join(transition_probabilities, by = "age_group") %>%
  mutate(FutureCount = InitialCount * TransitionProbability) %>%
  group_by(age_group, to_state) %>%
  summarize(FutureCount = sum(FutureCount))

# Add new entrants to the future states
final_population_2024 <- future_states_2024 %>%
  left_join(new_entrants_distribution, by = c("age_group", "to_state" = "state")) %>%
  mutate(FinalCount = FutureCount + coalesce(NewEntrantsByState, 0)) %>%
  select(age_group, state = to_state, Count = FinalCount)
